# IRG EDA

## Purpose

The purpose of this project is to conduct exploratory data analysis on the US housing market based on data from Zillow. Analysis will focus on home, and hte factors that influence the overall trend of the market: from a buyers to a sellers market. We will also look into the rents in the US, to determine whether an optimal strategy would be to BRR, or rent out a property, based on the location and profitability. IRG wants to focus on the Atlanta market, but is also interested in Philadelphia, Pa and Baltimore, Md.

# Introduction

In [2]:
# import libraries
import pandas as pd  
import plotly_express as px 
# import libraries
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np 
import pandas as pd 


In [3]:
# read home value forcast dataset
hvf = pd.read_csv('datasets\home_value_forcast.csv')

In [4]:
# first look at dataset
hvf.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,BaseDate,2023-02-28,2023-04-30,2024-01-31
0,102001,0,United States,country,NaN,2023-01-31,-0.1,0.0,0.5
1,394913,1,"New York, NY",msa,NY,2023-01-31,0.0,0.2,-1.3
2,753899,2,"Los Angeles, CA",msa,CA,2023-01-31,-1.3,-3.5,-3.3
3,394463,3,"Chicago, IL",msa,IL,2023-01-31,0.3,0.3,-1.8
4,394514,4,"Dallas, TX",msa,TX,2023-01-31,-0.4,-0.5,-0.4


In [5]:
# column information
hvf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RegionID    896 non-null    int64  
 1   SizeRank    896 non-null    int64  
 2   RegionName  896 non-null    object 
 3   RegionType  896 non-null    object 
 4   StateName   895 non-null    object 
 5   BaseDate    896 non-null    object 
 6   2023-02-28  884 non-null    float64
 7   2023-04-30  896 non-null    float64
 8   2024-01-31  896 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 63.1+ KB


In [6]:
# drop columns we do not need
hvf.drop(columns=['RegionID', 'RegionType'], inplace=True)

In [7]:
# rename columns
hvf.columns = ['size_rank', 'region', 'state', 'base_date', 'feb', 'apr', 'jan_24']

In [8]:
# check proper implementation 
hvf.columns

Index(['size_rank', 'region', 'state', 'base_date', 'feb', 'apr', 'jan_24'], dtype='object')

In [9]:
# list of states
states = ['NY', 'CA', 'IL', 'TX', 'VA', 'FL', 'PA', 'GA', 'AZ', 'MA',
       'MI', 'WA', 'MN', 'CO', 'MO', 'MD', 'NC', 'OR', 'NV', 'OH', 'IN',
       'TN', 'RI', 'WI', 'OK', 'LA', 'KY', 'UT', 'CT', 'AL', 'HI', 'NE',
       'SC', 'NM', 'ID', 'AR', 'IA', 'KS', 'MS', 'ME', 'DE', 'NH', 'AK',
       'NJ', 'SD', 'WV', 'ND', 'VT', 'MT', 'WY']
       
# sorted list of states
states_sorted = sorted(states)

In [10]:
hvf[hvf['state']=='GA']

,size_rank,region,state,base_date,feb,apr,jan_24
9,9,"Atlanta, GA",GA,2023-01-31,-0.3,-0.6,1.3
93,95,"Augusta, GA",GA,2023-01-31,0.2,0.5,1.6
136,138,"Savannah, GA",GA,2023-01-31,0.4,1.2,2.8
159,161,"Columbus, GA",GA,2023-01-31,0.1,0.0,0.3
198,201,"Macon, GA",GA,2023-01-31,0.6,1.4,3.4
209,213,"Athens, GA",GA,2023-01-31,0.4,1.0,3.4
215,219,"Gainesville, GA",GA,2023-01-31,-0.2,-0.3,1.8
228,232,"Warner Robins, GA",GA,2023-01-31,0.4,0.6,1.1
287,291,"Valdosta, GA",GA,2023-01-31,0.8,1.5,2.0
291,296,"Albany, GA",GA,2023-01-31,0.2,0.9,2.0


In [11]:
# mean and sum of metrics by state
state_sum = hvf.groupby('state')[['feb', 'apr', 'jan_24']].sum()
state_mean = hvf.groupby('state')[['feb', 'apr', 'jan_24']].mean()
state_feb = hvf.groupby('state')['feb'].sum()
state_apr = hvf.groupby('state')['apr'].sum()
state_jan = hvf.groupby('state')['jan_24'].sum()

Decide whether to use sum or mean

In [12]:
state_mean.sort_values(by='feb', ascending=False).head(10)

,feb,apr,jan_24
state,,,
MS,1.180000,2.295000,-0.295000
AR,1.073684,1.930000,0.895000
MI,1.035484,2.212903,2.600000
LA,0.982353,1.535294,-2.317647
KY,0.855000,1.860000,2.660000
TN,0.854167,1.965385,4.211538
AL,0.843750,1.245000,1.120000
TX,0.826984,1.477612,0.720896
IL,0.826923,1.403846,-0.792308


In [13]:
state_mean.index

Index(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA',
       'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO',
       'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK',
       'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI',
       'WV', 'WY'],
      dtype='object', name='state')

### Georgia Forcast

In [14]:
georgia = hvf.query("state== 'GA'")
penn = hvf.query("state== 'PA'")
maryl = hvf.query("state== 'MD'")

In [15]:
georgia.columns

Index(['size_rank', 'region', 'state', 'base_date', 'feb', 'apr', 'jan_24'], dtype='object')

In [16]:
georgia2 = georgia.set_index('region')

In [17]:
georgia2.columns

Index(['size_rank', 'state', 'base_date', 'feb', 'apr', 'jan_24'], dtype='object')

In [18]:
georgia2.head()

,size_rank,state,base_date,feb,apr,jan_24
region,,,,,,
"Atlanta, GA",9,GA,2023-01-31,-0.3,-0.6,1.3
"Augusta, GA",95,GA,2023-01-31,0.2,0.5,1.6
"Savannah, GA",138,GA,2023-01-31,0.4,1.2,2.8
"Columbus, GA",161,GA,2023-01-31,0.1,0.0,0.3
"Macon, GA",201,GA,2023-01-31,0.6,1.4,3.4


In [19]:
georgia2.drop(columns=['size_rank', 'state', 'base_date'], inplace=True)

In [20]:
px.scatter(georgia2, title='Georgia Home Value Forcecasts', template='plotly_white')

In [21]:
px.scatter(hvf, x='state', y=['feb', 'apr', 'jan_24'], opacity=.8)

In [22]:
px.bar(state_sum, title='Total Home Value Forcasts per State', template='simple_white')

In [23]:
px.bar(state_sum, title='Total Home Value Forcasts per State', template='plotly_white')

In [24]:
px.bar(state_mean, template='plotly_dark', orientation='v', height=900, color_discrete_sequence=['green', 'lightgreen', 'white'])

In [25]:
georgia.columns

Index(['size_rank', 'region', 'state', 'base_date', 'feb', 'apr', 'jan_24'], dtype='object')

In [26]:
georgia.reset_index(drop=True, inplace=True)
georgia.sort_values(by='jan_24', ascending=False).head(10)

,size_rank,region,state,base_date,feb,apr,jan_24
34,855,"Summerville, GA",GA,2023-01-31,2.1,5.3,10.2
31,839,"Thomaston, GA",GA,2023-01-31,0.2,1.4,6.9
33,844,"Toccoa, GA",GA,2023-01-31,0.1,1.1,6.0
26,667,"Cedartown, GA",GA,2023-01-31,0.0,0.8,5.8
30,807,"Jesup, GA",GA,2023-01-31,1.6,3.7,5.1
13,393,"Rome, GA",GA,2023-01-31,0.9,2.2,4.8
20,559,"Waycross, GA",GA,2023-01-31,1.2,2.6,4.6
32,840,"Bainbridge, GA",GA,2023-01-31,1.1,2.7,4.4
21,574,"Milledgeville, GA",GA,2023-01-31,0.5,1.6,3.6
4,201,"Macon, GA",GA,2023-01-31,0.6,1.4,3.4


In [27]:
px.bar(georgia, y=['feb', 'apr', 'jan_24'], x='region')

In [28]:
georgia

,size_rank,region,state,base_date,feb,apr,jan_24
0,9,"Atlanta, GA",GA,2023-01-31,-0.3,-0.6,1.3
1,95,"Augusta, GA",GA,2023-01-31,0.2,0.5,1.6
2,138,"Savannah, GA",GA,2023-01-31,0.4,1.2,2.8
3,161,"Columbus, GA",GA,2023-01-31,0.1,0.0,0.3
4,201,"Macon, GA",GA,2023-01-31,0.6,1.4,3.4
5,213,"Athens, GA",GA,2023-01-31,0.4,1.0,3.4
6,219,"Gainesville, GA",GA,2023-01-31,-0.2,-0.3,1.8
7,232,"Warner Robins, GA",GA,2023-01-31,0.4,0.6,1.1
8,291,"Valdosta, GA",GA,2023-01-31,0.8,1.5,2.0
9,296,"Albany, GA",GA,2023-01-31,0.2,0.9,2.0


In [29]:
px.box(georgia, y=['feb', 'apr', 'jan_24'], x=georgia.region, title='Georgia Regional Value Forecasts', template='plotly_white', color_discrete_sequence=['green', 'orange', 'white'], height=900, width=1400)

In [30]:
# multiple markers using dictionary jan_24 predictions
markers_dict = {'Atlanta, 1.3': [33.7490, -84.3880],
        'Augusta, 1.6': [33.4735, -82.0101],
        'Savannah, 2.8': [32.0809, -81.0912],
        'Columbus, 0.3': [32.4600, -84.9877],
        'Macon, 3.4': [32.8407, -83.6324],
        'Athens, 3.4': [33.9609, -83.3779],
        'Gainesville, 1.8': [34.2979, -83.8241],
        'Warner Robins, 1.1': [32.6130, -83.6242],
        'Valdosta, 2.0': [30.8327, -83.2785],
        'Albany, 2.0': [31.5785, -84.1557],
        'Dalton, 2.8': [34.7698, -84.9702],
        'Brunswick, 2.3': [31.1499, -81.4915],
        'LaGrange, 2.0': [33.0363, -85.0319],
        'Rome, 4.8': [34.2570, -85.1647],
        'Hinesville, 1.8': [31.8469, -81.5960],
        'Statesboro, 1.2': [32.4487, -81.7832],
        'Jefferson, 2.7': [34.1283, -83.6023],
        'Dublin, 3.1': [32.5404, -82.9038],
        'Calhoun, 3.1': [34.5034, -84.9427],
        'St. Marys, 0.7': [30.7563, -81.5729],
        'Waycross, 4.6': [31.2134, -82.3549],
        'Milledgeville, 3.6': [33.0801, -83.2321],
        'Douglas, 2.3': [31.5095, -82.8506],
        'Cornelia, 2.3': [34.5116, -83.5282],
        'Moultrie, 2.4': [31.1795, -83.7895],
        'Thomasville, 1.7': [30.8375, -83.9787],
        'Cedartown, 5.8 ': [34.0141, -85.2514],
        'Tifton, 0.3': [31.4505, -83.5086],
        'Vidalia, 2.8': [32.2177, -82.4128],
        'Americus, 0.6': [32.0724, -84.2329],
        'Jesup, 5.1': [31.6078, -81.8851],
        'Thomaston, 6.9': [32.8889, -84.3266],
        'Bainbridge, 4.4': [30.9034, -84.5820],
        'Toccoa, 6.0': [34.5793, -83.3326],
        'Summerville, 10.2': [34.4813, -85.3489],
        'Cordele, 0.6': [31.9635, -83.7820],
        'Fitzgerald, 2.1': [31.7174, -83.2527]
        }

# create map
map_cities = folium.Map(location=[33, -84], zoom_start=7, width=1200, height=900, control_scale=True)

# plot locations
for i in markers_dict.items():
    folium.Marker(location=i[1], popup=i[0], icon=folium.Icon(color='green', icon='car', prefix='fa')
    ).add_to(map_cities)

# measure control
measure_control = plugins.MeasureControl(position='topleft', active_color='blue', completed_color='green', primary_length_unit='miles')

# add measure control to map
map_cities.add_child(measure_control)
# display map
map_cities

In [31]:
hvf.head()

,size_rank,region,state,base_date,feb,apr,jan_24
0,0,United States,NaN,2023-01-31,-0.1,0.0,0.5
1,1,"New York, NY",NY,2023-01-31,0.0,0.2,-1.3
2,2,"Los Angeles, CA",CA,2023-01-31,-1.3,-3.5,-3.3
3,3,"Chicago, IL",IL,2023-01-31,0.3,0.3,-1.8
4,4,"Dallas, TX",TX,2023-01-31,-0.4,-0.5,-0.4


### Pennsylvania Forecast

In [32]:
penn2 = penn.set_index('region')
penn2.drop(columns=['size_rank', 'state', 'base_date'], inplace=True)

In [33]:
penn2.head()

,feb,apr,jan_24
region,,,
"Philadelphia, PA",0.2,0.3,-0.2
"Pittsburgh, PA",0.2,0.4,-0.6
"Allentown, PA",0.3,0.5,0.8
"Harrisburg, PA",0.2,0.0,-0.9
"Scranton, PA",1.3,3.0,3.0


In [34]:
px.scatter(penn2)

In [35]:
px.box(penn, y=['feb', 'apr', 'jan_24'], x=penn.region, title='Pennsylvania Regional Value Forecasts', template='plotly_white', color_discrete_sequence=['orange', 'orange', 'white'], height=900, width=1400)

In [36]:
# multiple markers using dictionary jan_24 predictions
markers_dict = {
'Philadelphia, -0.2': [39.9526, -75.1652], 
'Pittsburgh, -0.6': [40.4406, -79.9959], 
'Allentown, 0.8': [40.6084, -75.4902], 
'Harrisburg, -0.9': [40.2732, -76.8867], 
'Scranton, 3.0': [41.4080, -75.6624], 
'Lancaster, -0.3': [40.0379, -76.3055], 
'York, -1.2': [39.9626, -76.7277], 
'Reading, 0.7': [40.3356, -75.9277], 
'Erie, -0.8': [42.1292, -80.0851], 
'East Stroudsburg, 4.6': [41.0028, -75.1773], 
'State College, -1.7': [40.7934, -77.8600], 
'Chambersburg, 0.0': [39.9376, -77.6611], 
'Lebanon, -0.4': [40.3409, -76.4112], 
'Pottsville, 5.3': [40.6857, -76.1955], 
'Johnstown, 3.3': [40.3276, -78.9222], 
'Altoona, -1.9': [40.5187, -78.3947], 
'Williamsport, -2.9': [41.2412, -77.0011], 
'Gettysburg, -0.8': [39.8309, -77.2311], 
'Sunbury, 4.2': [40.8626, -76.7944], 
'New Castle, 1.4': [41.0037, -80.3470], 
'Meadville, 1.2': [41.6414, -80.1514], 
'Indiana, -2.0': [40.6215, -79.1528], 
'Bloomsburg, -0.6': [41.0047, -76.4549], 
'DuBois, 3.2': [41.1197, -78.7626], 
'Somerset, 0.8': [40.0054, -79.0781], 
'Sayre, -3.2': [41.9786, -76.5157], 
'Oil City, 0.9': [41.4309, -79.7076], 
'Lewistown, 2.8': [40.5993, -77.5712], 
'Huntingdon, -0.4': [40.4842, -78.0100], 
'Lewisburg, -1.1': [40.9645, -76.8844], 
'Bradford, 1.1': [41.9562, -78.6478], 
'Selinsgrove, 0.4': [40.7982, -76.8622], 
'Warren, -1.6': [41.8435, -79.1448], 
'Lock Haven, -3.0': [41.1373, -77.4469]}

# create map
map_cities = folium.Map(location=[41, -77], zoom_start=7, width=1200, height=900, control_scale=True)

# plot locations
for i in markers_dict.items():
    folium.Marker(location=i[1], popup=i[0], icon=folium.Icon(color='green', icon='car', prefix='fa')
    ).add_to(map_cities)

# measure control
measure_control = plugins.MeasureControl(position='topleft', active_color='blue', completed_color='green', primary_length_unit='miles')

# add measure control to map
map_cities.add_child(measure_control)
# display map
map_cities

### Maryland Forecasts

In [37]:
maryl2 = maryl.set_index('region')
maryl2.drop(columns=['size_rank', 'state', 'base_date'], inplace=True)

In [38]:
maryl2

,feb,apr,jan_24
region,,,
"Baltimore, MD",0.3,0.2,-1.6
"Hagerstown, MD",0.3,0.5,0.5
"California, MD",0.1,-0.1,-1.9
"Cumberland, MD",1.0,2.0,1.8
"Easton, MD",-0.1,-0.2,-1.4
"Cambridge, MD",0.8,1.6,2.3


In [39]:
px.scatter(maryl2, title='"old code.txt"')

In [40]:
px.box(maryl, y=['feb', 'apr', 'jan_24'], x=maryl.region, title='Maryland Regional Value Forecasts', template='plotly_dark', color_discrete_sequence=['whitesmoke', 'orange', 'white'], height=900, width=1400)

In [41]:
maryl.jan_24

21    -1.6
167    0.5
349   -1.9
394    1.8
717   -1.4
774    2.3
Name: jan_24, dtype: float64

In [42]:
# multiple markers using dictionary jan_24 predictions
markers_dict = {
    'Baltimore, -1.6': [39.2904, -76.6122],
    'Hagerstown, 0.5': [39.6418, -77.7200],
    'California, -1.9': [38.3001, -76.5075],
    'Cumberland, 1.8': [39.6528, -78.7625],
    'Easton, -1.4': [38.7745, -76.0768],
    'Cambridge, 2.3': [38.5633, -76.0768]
}

# create map
map_cities = folium.Map(location=[39, -76.6], zoom_start=7, width=1200, height=900, control_scale=True)

# plot locations
for i in markers_dict.items():
    folium.Marker(location=i[1], popup=i[0], icon=folium.Icon(color='green', icon='car', prefix='fa')
    ).add_to(map_cities)

# measure control
measure_control = plugins.MeasureControl(position='topleft', active_color='blue', completed_color='green', primary_length_unit='miles')

# add measure control to map
map_cities.add_child(measure_control)
# display map
map_cities

********************